In [14]:
%reload_ext autoreload
%autoreload 2

import torch
from data.names_dataset import NamesDataset
from torch.utils.data import DataLoader

# Initialize NamesDataset with the detected device
names_dataset = NamesDataset(data_folder="../datasets/names")


# train_dataset, test_dataset = torch.utils.data.random_split(names_dataset, [0.85, 0.15])
train_dataset, test_dataset = torch.utils.data.random_split(names_dataset, [0.85, 0.15])
print(f"Train dataset size: {len(train_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

self.index_to_token=[' ', "'", '-', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'Á', 'É', 'ß', 'à', 'á', 'ã', 'ä', 'ç', 'è', 'é', 'ê', 'ì', 'í', 'ñ', 'ò', 'ó', 'õ', 'ö', 'ù', 'ú', 'ü', 'ą', 'ł', 'ń', 'Ś', 'Ż', 'ż']
self.token_to_index={' ': 0, "'": 1, '-': 2, 'A': 3, 'B': 4, 'C': 5, 'D': 6, 'E': 7, 'F': 8, 'G': 9, 'H': 10, 'I': 11, 'J': 12, 'K': 13, 'L': 14, 'M': 15, 'N': 16, 'O': 17, 'P': 18, 'Q': 19, 'R': 20, 'S': 21, 'T': 22, 'U': 23, 'V': 24, 'W': 25, 'X': 26, 'Y': 27, 'Z': 28, 'a': 29, 'b': 30, 'c': 31, 'd': 32, 'e': 33, 'f': 34, 'g': 35, 'h': 36, 'i': 37, 'j': 38, 'k': 39, 'l': 40, 'm': 41, 'n': 42, 'o': 43, 'p': 44, 'q': 45, 'r': 46, 's': 47, 't': 48, 'u': 49, 'v': 50, 'w': 51, 'x': 52, 'y': 53, 'z': 54, 'Á': 55, 'É': 56, 'ß': 57, 'à': 58, 'á': 59, 'ã': 60, 'ä': 61, 

In [15]:
import time
import math
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence


class NamesClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NamesClassifier, self).__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            batch_first=False,
        )
        self.h2o = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(p=0.5)  # Add dropout regularization

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x is of shape (seq_length, batch_size, input_size)
        _lstm_output, (hidden, _cell) = self.lstm(x)
        # Use the last hidden state for classification
        last_hidden_state = hidden[-1]  # Shape: (batch_size, hidden_size)
        last_hidden_state = self.dropout(last_hidden_state)  # Apply dropout
        h2o_output = self.h2o(last_hidden_state)  # Shape: (batch_size, output_size)
        return h2o_output


def collate_fn(batch):
    inputs, labels = zip(*batch)
    inputs_padded = pad_sequence(list(inputs), batch_first=False, padding_side="left")
    labels = torch.stack(labels)
    return inputs_padded, labels


# # Updated train function to compute loss efficiently for the entire batch
# def train(
#     model: nn.Module,
#     dataloader: DataLoader,
#     optimizer: torch.optim.Optimizer,
#     criterion: nn.Module,
# ):
#     model.train()
#     total_loss = 0.0
#     correct = 0
#     total_samples = 0
#     for inputs, labels in dataloader:
#         # Forward pass for the entire batch
#         outputs = model(inputs)
#         loss = criterion(outputs, labels)

#         # Backward pass and optimization
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         predictions = outputs.argmax(dim=1)
#         correct += (predictions == labels).sum().item()
#         total_samples += labels.size(0)
#         total_loss += loss.item()

#     return total_loss / len(dataloader), correct / total_samples


# # Updated evaluate function to compute loss and accuracy efficiently
# def evaluate(model: nn.Module, dataloader: DataLoader, criterion: nn.Module):
#     model.eval()
#     total_loss = 0.0
#     correct = 0
#     total_samples = 0

#     with torch.no_grad():
#         for inputs, labels in dataloader:
#             # Forward pass for the entire batch
#             outputs = model(inputs)
#             loss = criterion(outputs, labels)

#             # Compute predictions and accuracy
#             predictions = outputs.argmax(dim=1)
#             correct += (predictions == labels).sum().item()
#             total_samples += labels.size(0)
#             total_loss += loss.item()

#     return total_loss / len(dataloader), correct / total_samples


# def fit(
#     model: nn.Module,
#     train_dataloader: DataLoader,
#     test_dataloader: DataLoader,
#     optimizer: torch.optim.Optimizer,
#     criterion: nn.Module,
#     num_epochs: int,
#     patience: int,
# ):
#     train_losses = []
#     train_accuracies = []
#     test_losses = []
#     test_accuracies = []
#     best_test_loss = math.inf
#     epochs_without_improvement = 0

#     for epoch in range(num_epochs):
#         start_time = time.perf_counter_ns()
#         train_loss, train_accuracy = train(
#             model, train_dataloader, optimizer, criterion
#         )
#         test_loss, test_accuracy = evaluate(model, test_dataloader, criterion)
#         end_time = time.perf_counter_ns()
#         elapsed_time = (end_time - start_time) / 1e9

#         train_losses.append(train_loss)
#         train_accuracies.append(train_accuracy)
#         test_losses.append(test_loss)
#         test_accuracies.append(test_accuracy)

#         print(
#             f"{epoch}  ({epoch / num_epochs:.0%}) "
#             f"\t{elapsed_time:.2f}s "
#             f"\tTrain Loss: {train_loss:.2f} "
#             f"\tTest Loss: {test_loss:.2f} "
#             f"\tTrain Accuracy: {train_accuracy:.2f} "
#             f"\tTest Accuracy: {test_accuracy:.2f}"
#         )

#         # Check for improvement in test loss
#         if test_loss < best_test_loss:
#             best_test_loss = test_loss
#             epochs_without_improvement = 0
#         else:
#             epochs_without_improvement += 1

#         # Stop training if no improvement for `patience` epochs
#         if epochs_without_improvement >= patience:
#             print("Early stopping triggered. Stopping training.")
#             break

#     return train_losses, train_accuracies, test_losses, test_accuracies

In [16]:
import matplotlib.pyplot as plt
from common.learner import Learner
from common.metrics import AccuracyMetric, ConfusionMatrixMetric, LossMetric

BATCH_SIZE = 64
LEARNING_RATE = 0.001
HIDDEN_SIZE = 64
NUM_EPOCHS = 50
PATIENCE = 5


train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=collate_fn,
)

test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=collate_fn,
)

num_classes = len(names_dataset.countries)


rnn = NamesClassifier(
    input_size=len(names_dataset.index_to_token),
    hidden_size=HIDDEN_SIZE,
    output_size=len(names_dataset.countries),
)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=LEARNING_RATE)

learner = Learner(
    model=rnn,
    optimizer=optimizer,
    criterion=criterion,
)

train_loss_metric = LossMetric(num_epochs=NUM_EPOCHS, batch_size=BATCH_SIZE)
train_accuracy_metric = AccuracyMetric(
    num_epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, num_classes=num_classes
)
train_confusion_matrix_metric = ConfusionMatrixMetric(
    num_epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, num_classes=num_classes
)
train_metrics = [
    train_accuracy_metric,
    train_confusion_matrix_metric,
]

eval_loss_metric = LossMetric(num_epochs=NUM_EPOCHS, batch_size=BATCH_SIZE)
eval_accuracy_metric = AccuracyMetric(
    num_epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, num_classes=num_classes
)
eval_confusion_matrix_metric = ConfusionMatrixMetric(
    num_epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, num_classes=num_classes
)
eval_metrics = [
    eval_accuracy_metric,
    eval_confusion_matrix_metric,
]


print("Starting training...")
learner.fit(train_dataloader, test_dataloader, NUM_EPOCHS, train_metrics, eval_metrics)
print("Training completed.")

plt.figure()
plt.plot(train_loss_metric.epoch_losses, label="Train Loss")
plt.plot(eval_loss_metric.epoch_losses, label="Test Loss")
plt.plot(train_accuracy_metric.epoch_corrects, label="Train Accuracy")
plt.plot(eval_accuracy_metric.epoch_corrects, label="Test Accuracy")
plt.xlabel("Epoch")
plt.legend()
plt.show()

Starting training...


ValueError: Expect outputs.shape == (267, 18), but got torch.Size([64, 18])

In [ ]:
def predict(model: nn.Module, input: str):
    model.eval()
    with torch.no_grad():
        base_tensor = names_dataset.name_to_tensor(input)
        for i in range(20):
            tensor = torch.cat(
                [
                    base_tensor,
                    torch.zeros((i, base_tensor.shape[1]), dtype=torch.float32),
                ],
                dim=0,
            )
            output = model(tensor)
            print(f"{tensor.shape=} {names_dataset.countries[output.argmax().item()]}")


print(predict(rnn, "Hai"))